<a href="https://colab.research.google.com/github/Sambit834/Assignment3/blob/main/assgn_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SAMBIT BANERJEE
g24ai1028
Assignment-4
Google Big Table

In [27]:
pip install google-cloud-bigtable

In [16]:
import os
import pandas as pd
from google.cloud import bigtable
from google.cloud.bigtable import column_family
from google.cloud.bigtable.row_filters import RowKeyRegexFilter


1.Write the method connect() to create a connection. Create a Bigtable data
 client and admin client

In [18]:
from google.cloud import bigtable

def connect(project_id, instance_id):
    # Create a Bigtable client
    client = bigtable.Client(project=project_id, admin=True)

    # Connect to the Bigtable instance
    instance = client.instance(instance_id)

    print(f"✅ Connected to Bigtable instance: {instance_id} in project: {project_id}")
    return instance


In [19]:
project_id = "disco-name-463612-m9"
instance_id = "ar2016"

instance = connect(project_id, instance_id)


✅ Connected to Bigtable instance: ar2016 in project: disco-name-463612-m9


 2. 10mark-Write the method createTable() to create a table to store the sensor data.

In [20]:
from google.cloud import bigtable
from google.cloud.bigtable import column_family

def connect(project_id, instance_id):
    client = bigtable.Client(project=project_id, admin=True)
    instance = client.instance(instance_id)
    print(f"✅ Connected to instance: {instance_id}")
    return instance

def create_table(instance, table_id="weather"):
    table = instance.table(table_id)

    if table.exists():
        print(f"⚠️ Table '{table_id}' already exists.")
    else:
        print(f"📦 Creating table '{table_id}'...")
        max_versions_rule = column_family.MaxVersionsGCRule(1)
        table.create(column_families={"sensor": max_versions_rule})
        print(f"✅ Table '{table_id}' created with column family 'sensor'.")

if __name__ == "__main__":
    project_id = "disco-name-463612-m9"
    instance_id = "ar2016"



In [21]:
import os
print(os.getcwd())  # Outputs the current working directory


/content


3. 5marks-Write the method load() to load the sensor data into the database. The data
 files are in the data folder.

In [22]:
from google.cloud.bigtable import row

def load_and_clean(filename, station_name):
    # Read the CSV file
    df = pd.read_csv(filename)
    # Add a 'Station' column for identification
    df['Station'] = station_name
    return df

df_pdx = load_and_clean("portland.csv", "Portland")
df_sea = load_and_clean("seatac.csv", "SeaTac")
df_yvr = load_and_clean("vancouver.csv", "Vancouver")

# Combine into one DataFrame
df_all = pd.concat([df_pdx, df_sea, df_yvr], ignore_index=True)

# Standardize column names for easier access
df_all.rename(columns={
    'Temperature': 'temperature',
    'Relhum': 'humidity',
    'Speed': 'wind_speed',
    'Pressure': 'pressure',
    'Date': 'date',
    'Time': 'time'
}, inplace=True)


 4. 10marks-Write the method query1() that returns the temperature at Vancouver on 2022-10-01 at 10 a.m.

In [23]:
def query1(df):
    """
    Returns the temperature at Vancouver on 2022-10-01 at 10:00.
    If not found, returns None and prints a message.
    """
    # Try to find the correct columns for date, time, temperature, and station
    # Based on your df_all, likely columns are:
    # 'Unnamed: 1' for Date, 'Unnamed: 2' for Time, 'Unnamed: 3' for Temperature, 'Station' for station name

    row = df[
        (df['Station'] == 'Vancouver') &
        (df['Unnamed: 1'] == '2022-10-01') &
        (df['Unnamed: 2'] == '10:00')
    ]
    if not row.empty:
        temp = row.iloc[0]['Unnamed: 3']
        print(f"Vancouver 2022-10-01 10:00 temperature: {temp}")
        return temp
    else:
        print("No data for Vancouver at 2022-10-01 10:00.")
        return None

# Example usage:
query1(df_all)

Vancouver 2022-10-01 10:00 temperature: 52


'52'

  5. 5marks-Write the method query2() that returns the highest wind speed in the month
 of September 2022 in Portland

In [24]:
def query2(df):
    """
    Returns the highest wind speed in Portland for September 2022.
    """
    mask = (
        (df['Station'] == 'Portland') &
        (df['Unnamed: 1'] >= '2022-09-01') & (df['Unnamed: 1'] <= '2022-09-30')
    )
    df2 = df[mask].copy()
    df2['Unnamed: 6'] = pd.to_numeric(df2['Unnamed: 6'], errors='coerce')
    if not df2.empty:
        max_speed = df2['Unnamed: 6'].max()
        max_row = df2[df2['Unnamed: 6'] == max_speed].iloc[0]
        print(f"Highest wind speed in Portland Sep 2022: {max_speed} at {max_row['Unnamed: 1']} {max_row['Unnamed: 2']}")
        return max_speed
    else:
        print("No Portland data for September 2022.")
        return None

# Example usage:
query2(df_all)

Highest wind speed in Portland Sep 2022: 25 at 2022-09-09 22:53


25

 6. 5marks-Write the method query3() that returns all the readings for SeaTac for October
 2, 2022

In [25]:
def query3(df):
    """
    Returns all readings for SeaTac for October 2, 2022.
    """
    mask = (df['Station'] == 'SeaTac') & (df['Unnamed: 1'] == '2022-10-02')
    readings = df[mask]
    if readings.empty:
        print("No readings found for SeaTac on 2022-10-02.")
        return None
    else:
        display(readings)
        return readings

# Example usage:
query3(df_all)

,Portland,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Station,SeaTac Airport,Vancouver
21616,NaN,2022-10-02,0:53,74,53,47.8,9,M,1014.1,SeaTac,2459855.037,NaN
21617,NaN,2022-10-02,1:53,69,53,56.7,7,M,1014.1,SeaTac,2459855.078,NaN
21618,NaN,2022-10-02,2:53,67,53,60.7,7,M,1014.3,SeaTac,2459855.12,NaN
21619,NaN,2022-10-02,3:53,66,53,62.9,7,M,1014.4,SeaTac,2459855.162,NaN
21620,NaN,2022-10-02,4:53,64,53,67.4,7,M,1014.2,SeaTac,2459855.203,NaN
21621,NaN,2022-10-02,5:53,63,52,67.3,7,M,1014.1,SeaTac,2459855.245,NaN
21622,NaN,2022-10-02,6:53,61,52,72.2,8,M,1014.3,SeaTac,2459855.287,NaN
21623,NaN,2022-10-02,7:53,63,51,64.8,9,M,1014.2,SeaTac,2459855.328,NaN
21624,NaN,2022-10-02,8:53,61,53,74.9,4,M,1014,SeaTac,2459855.37,NaN
21625,NaN,2022-10-02,9:53,59,52,77.5,0,M,1014.2,SeaTac,2459855.412,NaN


,Portland,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Station,SeaTac Airport,Vancouver
21616,NaN,2022-10-02,0:53,74,53,47.8,9,M,1014.1,SeaTac,2459855.037,NaN
21617,NaN,2022-10-02,1:53,69,53,56.7,7,M,1014.1,SeaTac,2459855.078,NaN
21618,NaN,2022-10-02,2:53,67,53,60.7,7,M,1014.3,SeaTac,2459855.12,NaN
21619,NaN,2022-10-02,3:53,66,53,62.9,7,M,1014.4,SeaTac,2459855.162,NaN
21620,NaN,2022-10-02,4:53,64,53,67.4,7,M,1014.2,SeaTac,2459855.203,NaN
21621,NaN,2022-10-02,5:53,63,52,67.3,7,M,1014.1,SeaTac,2459855.245,NaN
21622,NaN,2022-10-02,6:53,61,52,72.2,8,M,1014.3,SeaTac,2459855.287,NaN
21623,NaN,2022-10-02,7:53,63,51,64.8,9,M,1014.2,SeaTac,2459855.328,NaN
21624,NaN,2022-10-02,8:53,61,53,74.9,4,M,1014,SeaTac,2459855.37,NaN
21625,NaN,2022-10-02,9:53,59,52,77.5,0,M,1014.2,SeaTac,2459855.412,NaN


 7. 5marks-Write the method query4() that returns the highest temperature at any station
 in the summer months of 2022 (July (7), August (8))

In [26]:
def query4(df):
    """
    Returns the highest temperature at any station in July or August 2022.
    """
    # Filter for July and August 2022
    mask = (
        (df['Unnamed: 1'] >= '2022-07-01') & (df['Unnamed: 1'] <= '2022-08-31')
    )
    df_summer = df[mask].copy()
    # Convert temperature column to numeric
    df_summer['Unnamed: 3'] = pd.to_numeric(df_summer['Unnamed: 3'], errors='coerce')
    if not df_summer.empty:
        max_temp = df_summer['Unnamed: 3'].max()
        max_row = df_summer[df_summer['Unnamed: 3'] == max_temp].iloc[0]
        print(f"Highest temperature in summer 2022: {max_temp} at {max_row['Station']} on {max_row['Unnamed: 1']} {max_row['Unnamed: 2']}")
        return max_temp
    else:
        print("No data for summer 2022.")
        return None

# Example usage:
query4(df_all)

Highest temperature in summer 2022: 101 at Portland on 2022-07-26 23:53


101